# Retrieval-Augmented Generation (RAG) System for Financial Reports

# STEP 2: Structured Data Integration

### Objective: Build a simple RAG pipeline for factual QA from a single financial report.

In [30]:
import warnings
warnings.filterwarnings("ignore")



In [31]:
!pip install PyMuPDF
!pip install tabula-py
!pip install pdfplumber

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Preprocessing: Extract and clean text from PDF.

In [32]:
import fitz

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

raw_text = extract_text_from_pdf("/kaggle/input/financial-reporta/Metas Q1 2024 Financial Report.pdf")

### Table Extraction: Parse tables into structured formats (e.g., DataFrame,JSON).

In [33]:
import pdfplumber
import pandas as pd

tables = []
with pdfplumber.open("/kaggle/input/financial-reporta/Metas Q1 2024 Financial Report.pdf") as pdf:
    for page in pdf.pages:
        page_tables = page.extract_tables()
        for table in page_tables:
            if table and len(table) > 1:
                df = pd.DataFrame(table[1:], columns=table[0])
                tables.append(df)

### Chunking & Embedding: Split into chunks; generate embeddings with an open-source model.


In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=200)
chunks = splitter.split_text(raw_text)

In [35]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(chunks)

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

In [36]:
pip install faiss-cpu


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


### Retrieval: Use vector similarity to retrieve top-3 relevant chunks.

In [37]:

import faiss
import numpy as np

dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))


query = "What was the revenue for the first quarter of 2024?"
query_embedding = embedding_model.encode([query]).astype('float32')
top_k = 3
D, I = index.search(np.array(query_embedding), top_k)
retrieved_chunks = [chunks[i] for i in I[0]]
context = "\n".join(retrieved_chunks)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
def remove_newlines(text):
    if isinstance(text, list):
        text = ' '.join(text)
    return text.replace('\n', ' ')



cleaned = remove_newlines(context)
print(cleaned)

First Quarter 2024 Financial Highlights Three Months Ended March 31, % Change In millions, except percentages and per share amounts 2024 2023 Revenue $  36,455  $  28,645   27 % Costs and expenses 2024 2023 Revenue $  36,455  $  28,645   27 % Costs and expenses   22,637    21,418   6 % Income from operations $  13,818  $  7,227   91 % Operating margin  38 %  25 % Provision for income taxes $ 1 CFO Outlook Commentary We expect second quarter 2024 total revenue to be in the range of $36.5-39 billion. Our guidance assumes foreign currency is a


### Hybrid Retrieval: Combine vector search (text) + keyword/SQL-like search(structured).

In [39]:
def search_tables_simple(query, tables):
    if not tables:
        return None

    stop_words = ['what', 'was', 'the', 'in', 'to', 'and', 'or', 'a', 'an', 'is', 'are', 'compared']
    words = [word.lower().strip('?.,!') for word in query.split()
             if word.lower() not in stop_words and len(word) > 2]

    print(f"Searching for: {words}")

    results = []
    for i, table in enumerate(tables):
        matching_rows = []


        for idx, row in table.iterrows():

            row_text = ' '.join(str(val).lower() for val in row.values)


            if any(word in row_text for word in words):
                matching_rows.append(idx)

        if matching_rows:
            matches = table.loc[matching_rows]
            print(f"Table {i+1}: {len(matches)} matches")
            results.append((i+1, matches))

    return results if results else None



results = search_tables_simple(query, tables)

if results:
    for table_num, matches in results:
        print(f"\n--- Table {table_num} matches ---")
        print(matches)
else:
    print("No matches found")

Searching for: ['revenue', 'for', 'first', 'quarter', '2024']
Table 1: 1 matches
Table 2: 3 matches
Table 4: 1 matches
Table 5: 2 matches
Table 7: 2 matches
Table 8: 9 matches

--- Table 1 matches ---
                    Revenue $ 36,455 $ 28,645 27 %
3  Provision for income taxes $ 1,814 $ 1,598 14 %

--- Table 2 matches ---
                                    Revenue $ 36,455 $ 28,645
1                           Cost of revenue       6,640 6,108
8  Income before provision for income taxes      14,183 7,307
9                Provision for income taxes       1,814 1,598

--- Table 4 matches ---
                Cash flows from operating activities
5  Impairment charges for facilities consolidatio...

--- Table 5 matches ---
                         Supplemental cash flow data
0        Cash paid for income taxes, net $ 630 $ 405
1  Cash paid for interest, net of amounts capital...

--- Table 7 matches ---
                          Revenue:  None  None
1            Other revenue 380 205  N

In [40]:
results=search_tables_simple(query, tables)

Searching for: ['revenue', 'for', 'first', 'quarter', '2024']
Table 1: 1 matches
Table 2: 3 matches
Table 4: 1 matches
Table 5: 2 matches
Table 7: 2 matches
Table 8: 9 matches


### Generation: Answer queries using an open-source LLM

In [41]:
print(query)
print(context)
print(results)

What was the revenue for the first quarter of 2024?
First Quarter 2024 Financial Highlights
Three Months Ended March 31,
% Change
In millions, except percentages and per share amounts
2024
2023
Revenue
$ 
36,455 
$ 
28,645 
 27 %
Costs and expenses
2024
2023
Revenue
$ 
36,455 
$ 
28,645 
 27 %
Costs and expenses
 
22,637 
 
21,418 
 6 %
Income from operations
$ 
13,818 
$ 
7,227 
 91 %
Operating margin
 38 %
 25 %
Provision for income taxes
$
1
CFO Outlook Commentary
We expect second quarter 2024 total revenue to be in the range of $36.5-39 billion. Our guidance assumes foreign currency is a
[(1,                     Revenue $ 36,455 $ 28,645 27 %
3  Provision for income taxes $ 1,814 $ 1,598 14 %), (2,                                     Revenue $ 36,455 $ 28,645
1                           Cost of revenue       6,640 6,108
8  Income before provision for income taxes      14,183 7,307
9                Provision for income taxes       1,814 1,598), (4,                 Cash flows from op

In [42]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Ensure padding token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Example context + query
context = cleaned
query = query

# Instruction-style prompt
prompt = (
    f"<|user|>\n"
    f"Answer the query:\n{query}\n\n"
    f"Based on the following:\n"
    f"- Text context:\n{context}\n\n"
    f"- Structured data:\n{results}\n"
    f"<|assistant|>"
)



# Tokenize
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=1024)

# Generate
outputs = model.generate(
    **inputs,
    max_new_tokens=130,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

# Decode only new tokens (exclude the prompt)
input_length = inputs["input_ids"].shape[-1]
generated_ids = outputs[0][input_length:]
answer = tokenizer.decode(generated_ids, skip_special_tokens=True)

print("Answer:", answer)


Answer: 
To answer the query, we need to analyze the structured data provided. The revenue for the first quarter of 2024 is $36,455 million.


#### Model works fine

### Creating a function

In [43]:


import faiss
import fitz  # PyMuPDF
import numpy as np
import pandas as pd
import pdfplumber
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("Starting one-time setup...")


path = "/kaggle/input/financial-reporta/Metas Q1 2024 Financial Report.pdf" 
raw_text = ""
try:
    doc = fitz.open(path)
    for page in doc:
        raw_text += page.get_text()
    doc.close()
    print(f"Successfully extracted text from '{path}'.")
except Exception as e:
    print(f"Error reading PDF: {e}")
    
    raw_text = "" 

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=200)
chunks = splitter.split_text(raw_text)
print(f"Split text into {len(chunks)} chunks.")


embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
dimension = embedding_model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(dimension)

print("Embedding text chunks...")
chunk_embeddings = embedding_model.encode(chunks, show_progress_bar=True)
index.add(chunk_embeddings)
print("FAISS index is built and populated.")

model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Setup complete.")


class PDFQueryProcessor:
    def __init__(self, path, embedding_model, index, chunks, tokenizer, model):
        self.path = path
        self.embedding_model = embedding_model
        self.index = index
        self.chunks = chunks
        self.tokenizer = tokenizer
        self.model = model
     
        self.tables = self._extract_tables()

    def _extract_tables(self):
        tables = []
        with pdfplumber.open(self.path) as pdf:
            for page in pdf.pages:
                page_tables = page.extract_tables()
                for table in page_tables:
                    if table and len(table) > 1:
                        df = pd.DataFrame(table[1:], columns=table[0])
                        tables.append(df)
        return tables
    
    

    def _search_tables_simple(self, query):

        stop_words = {'what', 'was', 'the', 'in', 'to', 'and', 'or', 'a', 'an', 'is', 'are', 'compared'}
        words = [word.lower().strip('?.,!') for word in query.split() if word.lower() not in stop_words and len(word) > 2]
        
        results = []
        for i, table in enumerate(self.tables):
            table_str = table.astype(str).apply(lambda x: ' '.join(x).lower(), axis=1)
            matching_rows_mask = table_str.str.contains('|'.join(words), case=False, na=False)
            
            if matching_rows_mask.any():
                matches = table[matching_rows_mask]
                results.append((i+1, matches))
        return results if results else None
        
    def _format_results_for_prompt(self, table_results):
        if not table_results:
            return "No relevant structured data found."
        
        formatted_string = "Structured data:\n"
        for i, (table_num, df) in enumerate(table_results):
            formatted_string += f"--- Relevant Data from Table {table_num} ---\n"
            formatted_string += df.to_markdown(index=False)
            formatted_string += "\n"
        return formatted_string


    def get_answer(self, query):

        query_embedding = self.embedding_model.encode([query])


        top_k = 3
        _, I = self.index.search(query_embedding, top_k)
        retrieved_chunks = [self.chunks[i] for i in I[0]]
        text_context = "\n".join(retrieved_chunks)


        table_results = self._search_tables_simple(query)
        formatted_tables = self._format_results_for_prompt(table_results)


        prompt = (
    f"<|user|>\n"
    f"Answer the query in short:\n{query}\n\n"
    f"Based on the following:\n"
    f"- Text context: {text_context}\n"
    f"- Structured data: {formatted_tables}\n"
    f"<|assistant|>"
)



        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048)
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=250,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=self.tokenizer.eos_token_id,
        )

        input_length = inputs["input_ids"].shape[-1]
        answer = self.tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)

        print(answer)
        return answer


processor = PDFQueryProcessor(
    path=path,
    embedding_model=embedding_model,
    index=index,
    chunks=chunks,
    tokenizer=tokenizer,
    model=model
)




Starting one-time setup...
Successfully extracted text from '/kaggle/input/financial-reporta/Metas Q1 2024 Financial Report.pdf'.
Split text into 316 chunks.
Embedding text chunks...


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

FAISS index is built and populated.
Setup complete.


## Testing the model


In [44]:
answer1 = processor.get_answer("What was Meta’s net income in Q1 2024 compared to Q1 2023?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


The net income in Q1 2024 compared to Q1 2023 was $12,369 compared to $5,709.


In [45]:
answer2 = processor.get_answer("Summarize Meta’s operating expenses in Q1 2024.")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Based on the provided text, the operating expenses for Meta in Q1 2024 were $19,246 million.
